In [ ]:
import geopandas as gpd
import pystac_client
import planetary_computer
import rioxarray as rio
import xarray as xr
from itertools import product
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib_scalebar.scalebar import ScaleBar
import numpy as np
from shapely import Point, box
import osmnx as ox
import cartopy.crs as ccrs
from osgeo import gdal
from tqdm import tqdm
from rasterio.features import geometry_mask
plt.rcParams['font.family'] = 'DejaVu Sans Mono'

### get national park boundary and peaks

In [ ]:
### get national park boundary and peaks within national park
cgnp_4326 = ox.features.features_from_place('Cairngorms National Park',
                                       tags={'boundary':'protected area',
                                             'designation':'national_park'}).reset_index()

boundary = cgnp_4326.loc[0,'geometry']
aoi = box(*boundary.bounds)

peaks_4326 = ox.features.features_from_polygon(boundary,
                                          tags={'natural':'peak'}).reset_index()

peaks_4326['ele'] = peaks_4326['ele'].astype('float32')

# get local utm and transform to projected coordinate system
prj = ccrs.epsg(cgnp_4326.estimate_utm_crs().to_epsg())

cgnp = cgnp_4326.to_crs(prj)
peaks = peaks_4326.to_crs(prj)

### get DEM from planetary computer

In [ ]:
### get elevation data for national park's bounding box
# and export to gdal/dem.tif so don't need to re=run

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

search = catalog.search(collections=['cop-dem-glo-30'],
                        intersects=aoi)
items = list(search.items())
tmps = []
for item in items:
    signed_asset = planetary_computer.sign(item.assets["data"])
    with rio.open_rasterio(signed_asset.href) as tmp:
        tmps.append(tmp.squeeze().drop_vars("band").rename('z'))
        tmp.close()
        
dem_4326 = ((xr.merge(tmps))
            .rio.set_crs(4326)
            .rio.write_transform()
            .rio.clip_box(*aoi.bounds)
            .rio.write_transform())

dem = dem_4326.rio.reproject(prj, resolution=30, nodata=np.nan)['z']

dem.rio.to_raster('gdal/dem.tif', driver='GTiff', recalc_transform=True)

### get viewsheds

In [ ]:
def run_viewshed(p, id):
    '''
    p: shapely point geometry
    id: int a unique identifer
    '''
    xyz = [p.x, p.y, 1.0]    
    ds = gdal.ViewshedGenerate(
        srcBand = src_ds.GetRasterBand(1),
        driverName = "MEM",
        # targetRasterName = f"viewsheds/view_{id}.tif",
        targetRasterName = str(id),
        creationOptions = ["INTERLEAVE=BAND"],
        observerX = xyz[0],
        observerY = xyz[1],
        observerHeight = xyz[2],
        targetHeight = 0,
        visibleVal = 255,
        invisibleVal = 0,
        outOfRangeVal = 0,
        noDataVal = 0,
        dfCurvCoeff = 0.85714,
        mode = gdal.GVM_Edge,
        maxDistance = 80_000,
        callback = 0,
        callback_data = None,
        heightMode = gdal.GVOT_NORMAL,
        options=['UNUSED=YES']
        )
    return ds

dss = []
src_ds = gdal.Open('gdal/dem.tif')
for row in tqdm(peaks.itertuples()):
    ds = run_viewshed(row.geometry, row.Index)
    dss.append(ds)
    
vrt_options = gdal.BuildVRTOptions(resampleAlg='cubic', separate=True)
vrt = gdal.BuildVRT('gdal/my.vrt', dss, options=vrt_options)
gdal.Translate('gdal/all_viewsheds.tif', vrt, format='GTiff')

In [ ]:
## read in results and clip to national park

vs = rio.open_rasterio('gdal/all_viewsheds.tif')
all_vs = vs.sum(dim='band')/255

dimensions = dict(zip(all_vs.dims, all_vs.shape))

### mask to national park boundary
geomask = geometry_mask([cgnp['geometry'][0]],
                        out_shape=(dimensions['y'], dimensions['x']),
                        transform=all_vs.rio.transform(),
                        invert=True)
geomask = xr.DataArray(geomask, dims=['y','x'])
masked = all_vs.where(geomask==True)

In [ ]:
## identify least / most visible regions
idx_max = masked.argmax(['y','x'])
idx_min = masked.argmin(['y','x'])

most_visible = gpd.GeoSeries(Point(masked.isel(idx_max).x,
                                   masked.isel(idx_max).y),
                             crs=prj)

peaks.loc[peaks.apply(lambda q: q.geometry.distance(most_visible),
                      axis=1).idxmin()]

least_visible = gpd.GeoSeries(Point(masked.isel(idx_min).x,
                                    masked.isel(idx_min).y),
                             crs=prj)

### plotting

In [ ]:
fig, ax = plt.subplots(figsize=[10,10],
                       subplot_kw={'projection':prj})
masked.plot(cmap='magma', norm=LogNorm(), vmin=1, add_colorbar=False)


cgnp.plot(ax=ax, fc='none', ec='w', linewidth=2)

minx,miny,maxx,maxy = cgnp.total_bounds
ax.set_xlim(minx-5000,maxx+5000)
ax.set_ylim(miny-5000,maxy+5000)

ax.add_artist(ScaleBar(dx=1))

ax.set_facecolor('k')
ax.set_title('')
ax.set_title('visibility\n in the\n  cairngorms',
             loc='left',
             y=0.8,
             x=0.08,
             color='w')

ax.annotate(text='Data:\nOpenStreetMap\nCopernicus Global Digital Elevation Model (ESA)',
            xy=(0.99,0.01), 
            xycoords='axes fraction', 
            ha='right', 
            fontsize=8,
            c='w')

ax.annotate(text='by:tlohde',
            xy=(0.01,0.01), 
            xycoords='axes fraction',
            ha='left',
            fontsize=8,
            c='w')

fig.savefig('day20.png', dpi=300, bbox_inches='tight')